In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np

from inversion_utils import *
import pickle
from sklearn.model_selection import train_test_split

In [3]:
SEED = 0

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

torch.backends.cudnn.benchmark = True 
torch.backends.cuda.matmul.allow_tf32 = True

LLM = namedtuple('LLM', ['language_model', 'tokenizer', 'processor', 'name', 'model_type'])

In [4]:
model_type = 'llama'
# model_type = 'qwen'

# MODEL_VERSION = '3'
MODEL_VERSION = '3.1'
# MODEL_VERSION = '3.3'

MODEL_SIZE = '8B'
# MODEL_SIZE = '70B'

llm = select_llm(model_type, MODEL_VERSION=MODEL_VERSION, MODEL_SIZE=MODEL_SIZE)

Loading meta-llama/Meta-Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ODE (wrong)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchdiffeq import odeint_adjoint as odeint
import numpy as np
from copy import deepcopy
import logging

In [8]:
with open("../data/moods/all_antonym_pairs.pkl", 'rb') as file:
    all_e = pickle.load(file)

In [9]:
# with open("inversion_matrices/llama8b/big_trans_mat.pkl", 'rb') as file:
#     lrr = pickle.load(file)

In [10]:
str_X = [i[0] for i in all_e]
str_Y = [i[1] for i in all_e]

In [11]:
print(str_X)
print(len(str_X))
print(str_Y)

['optimistic', 'enthusiastic', 'energetic', 'tense', 'confident', 'bold', 'stern', 'cold', 'cynical', 'hopeful', 'ecstatic', 'triumphant', 'lighthearted', 'jovial', 'dramatic', 'earnest', 'boastful', 'cheerful', 'joyful', 'content', 'satisfied', 'pleased', 'delighted', 'excited', 'thrilled', 'elated', 'euphoric', 'blissful', 'serene', 'peaceful', 'calm', 'tranquil', 'patient', 'accepting', 'loving', 'affectionate', 'compassionate', 'sympathetic', 'kind', 'generous', 'friendly', 'sociable', 'outgoing', 'animated', 'vibrant', 'spirited', 'motivated', 'ambitious', 'determined', 'assertive', 'courageous', 'brave', 'fearless', 'secure', 'assured', 'certain', 'decisive', 'resolute', 'stubborn', 'serious', 'solemn', 'melancholic', 'distressed', 'worried', 'happy', 'angry', 'grateful', 'humble', 'proud', 'embarrassed', 'guilty', 'remorseful', 'regretful', 'sentimental', 'emotional', 'expressive', 'spontaneous', 'impulsive', 'reckless', 'wild', 'manic', 'hyperactive', 'vigorous', 'robust', 'har

In [12]:
path = '../directions_moods_plus_llama/'

X = [just_dirs(llm, i, path=path) for i in str_X]
Y = [just_dirs(llm, i, path=path) for i in str_Y]

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

D

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components

In [13]:
print(len(X))
print(X[10].keys())
print(X[10][-10].shape)

239
dict_keys([-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31])
torch.Size([1, 4096])


In [13]:
def setup_logger(log_file='training.log'):
    """
    Setup logger to write to file and console
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()  # Also print to console
        ]
    )

In [14]:
class TransformDataset(Dataset):
    def __init__(self, inputs, outputs):
        """
        inputs: numpy array or tensor of shape (n_samples, 4096)
        outputs: numpy array or tensor of shape (n_samples, 4096)
        """
        self.inputs = inputs
        self.outputs = outputs
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

In [6]:
# ODE Function
class ODEFunc(nn.Module):
    def __init__(self, hidden_dim=4096):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
    def forward(self, t, y):
        return self.net(y)

# Neural ODE Model
class NeuralODEModel(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=4096):
        super(NeuralODEModel, self).__init__()
        self.encoder = nn.Linear(input_dim, hidden_dim)
        self.ode_func = ODEFunc(hidden_dim)
        self.decoder = nn.Linear(hidden_dim, input_dim)
        
    def forward(self, x):
        # Encode input
        h = self.encoder(x)
        
        # Solve ODE from t=0 to t=1
        t = torch.tensor([0.0, 1.0]).to(x.device)
        h = odeint(self.ode_func, h, t, method='dopri5')[1]
        
        # Decode output
        output = self.decoder(h)
        return output

In [7]:
def cosine_loss(predictions, targets):
    return 1 - F.cosine_similarity(predictions, targets, dim=1).mean()

In [17]:
def train_model(model, train_loader, val_loader=None, num_epochs=5, lr=0.001, print_every=1, log_file='training.log'):
    
    setup_logger(log_file)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    # Track best validation loss and model state
    best_val_loss = float('inf')
    best_model_state = None

    logging.info(f'Starting training on device: {device}')
    logging.info(f'Number of epochs: {num_epochs}, Learning rate: {lr}')
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        
        for batch_inputs, batch_outputs in train_loader:
            batch_inputs = batch_inputs.to(device)
            batch_outputs = batch_outputs.to(device)

            optimizer.zero_grad()
            predictions = model(batch_inputs)
            # loss = criterion(predictions, batch_outputs)
            loss = cosine_loss(predictions, batch_outputs)
            
            # print(f"train loss {loss}")
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation phase (if validation loader provided)
        if val_loader is not None:
            model.eval()
            val_loss = 0
            
            with torch.no_grad():
                for batch_inputs, batch_outputs in val_loader:
                    batch_inputs = batch_inputs.to(device)
                    batch_outputs = batch_outputs.to(device)
                    
                    predictions = model(batch_inputs)
                    # loss = criterion(predictions, batch_outputs)
                    loss = cosine_loss(predictions, batch_outputs)

                    # print(f"val loss {loss}")

                    val_loss += loss.item()
            
            avg_val_loss = val_loss / len(val_loader)
            
            # Save best model state
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_model_state = deepcopy(model.state_dict())
            
            # Print losses
            if (epoch + 1) % print_every == 0:
                logging.info(f'Epoch [{epoch+1}/{num_epochs}], '
                           f'Train Loss: {avg_train_loss:.6f}, '
                           f'Val Loss: {avg_val_loss:.6f}')
        else:
            # Print only training loss if no validation set
            if (epoch + 1) % print_every == 0:
                logging.info(f'Epoch [{epoch+1}/{num_epochs}], '
                           f'Train Loss: {avg_train_loss:.6f}')
    
    # Load best model state if validation was used
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        logging.info(f'Best model loaded with validation loss: {best_val_loss:.6f}')
    
    logging.info('Training completed!')
    return model

In [14]:
train_indices, temp_indices = train_test_split(np.arange(len(X)), test_size=0.3, random_state=SEED)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.3333, random_state=SEED)

In [15]:
print(np.array(str_X)[test_indices])
print(np.array(str_Y)[test_indices])

['burdened' 'lighthearted' 'unpredictable' 'aloof' 'crushed' 'stiff'
 'alarmed' 'concerned' 'vibrant' 'fearless' 'offended' 'insincere'
 'possessive' 'hectic' 'focused' 'weighed-down' 'infuriated' 'interested'
 'earnest' 'confrontational' 'forlorn' 'mindful' 'evasive' 'remorseful']
['unburdened' 'somber' 'reliable' 'warm' 'uplifted' 'loose' 'unalarmed'
 'unconcerned' 'dull' 'fearful' 'flattered' 'sincere' 'sharing'
 'leisurely' 'distracted' 'buoyant' 'soothed' 'uninterested' 'flippant'
 'diplomatic' 'connected' 'mindless' 'direct' 'unrepentant']


In [18]:
# [{}, {}, ...*239] -> {[], [], ...*32}

In [19]:
layers = X[0].keys()

X_dict_temp = {i: [] for i in layers}
Y_dict_temp = {i: [] for i in layers}

for inp, out in zip(X, Y):
    for layer in layers:
        X_dict_temp[layer].append(inp[layer])
        Y_dict_temp[layer].append(out[layer])
    

X_dict = {i: torch.cat(X_dict_temp[i]) for i in X_dict_temp}
Y_dict = {i: torch.cat(Y_dict_temp[i]) for i in Y_dict_temp}
# X_dict = {i: torch.cat(X_dict_temp[i]).to("cuda") for i in X_dict_temp}
# Y_dict = {i: torch.cat(Y_dict_temp[i]).to("cuda") for i in Y_dict_temp}

print(X_dict[-1].shape)
print(Y_dict[-1].shape)

torch.Size([239, 4096])
torch.Size([239, 4096])


In [20]:
X_train = {layer: X_dict[layer][train_indices] for layer in X_dict}
X_val = {layer: X_dict[layer][val_indices] for layer in X_dict}
X_test = {layer: X_dict[layer][test_indices] for layer in X_dict}

Y_train = {layer: Y_dict[layer][train_indices] for layer in Y_dict}
Y_val = {layer: Y_dict[layer][val_indices] for layer in Y_dict}
Y_test = {layer: Y_dict[layer][test_indices] for layer in Y_dict}

In [21]:
# Normalize the data
# X_mean, X_std = X_train.mean(axis=0), X_train.std(axis=0) + 1e-8
# Y_mean, Y_std = Y_train.mean(axis=0), Y_train.std(axis=0) + 1e-8

# X_normalized = (X_train - X_mean) / X_std
# Y_normalized = (Y_train - Y_mean) / Y_std

In [22]:
X_mean = {layer: X_train[layer].mean(axis=0) for layer in X_train}
X_std = {layer: X_train[layer].std(axis=0) + 1e-8 for layer in X_train}

Y_mean = {layer: Y_train[layer].mean(axis=0) for layer in Y_train}
Y_std = {layer: Y_train[layer].std(axis=0) + 1e-8 for layer in Y_train}

In [23]:
X_train_normalized = {layer: (X_train[layer] - X_mean[layer]) / X_std[layer] for layer in X_train}
X_val_normalized = {layer: (X_val[layer] - X_mean[layer]) / X_std[layer] for layer in X_val}
X_test_normalized = {layer: (X_test[layer] - X_mean[layer]) / X_std[layer] for layer in X_test}

Y_train_normalized = {layer: (Y_train[layer] - Y_mean[layer]) / Y_std[layer] for layer in Y_train}
Y_val_normalized = {layer: (Y_val[layer] - Y_mean[layer]) / Y_std[layer] for layer in Y_val}
Y_test_normalized = {layer: (Y_test[layer] - Y_mean[layer]) / Y_std[layer] for layer in Y_test}

In [24]:
print(X_train_normalized[-10].shape)
print(X_val_normalized[-10].shape)
print(X_test_normalized[-10].shape)

print(Y_train_normalized[-10].shape)
print(Y_val_normalized[-10].shape)
print(Y_test_normalized[-10].shape)

torch.Size([167, 4096])
torch.Size([48, 4096])
torch.Size([24, 4096])
torch.Size([167, 4096])
torch.Size([48, 4096])
torch.Size([24, 4096])


In [25]:
# X_layer = fetch_layer(X, layer) # 239*4096
# Y_layer = fetch_layer(Y, layer) # 239*4096

# # ToDo: Try to normalize to std 1

# X_train_split = X_layer[train_indices]
# Y_train_split = Y_layer[train_indices]

# X_val_split = X_layer[val_indices]
# Y_val_split = Y_layer[val_indices]

# X_test_split = X_layer[test_indices]
# Y_test_split = Y_layer[test_indices]


# train_dataset = TransformDataset(X_train_split, Y_train_split)
# val_dataset = TransformDataset(X_val_split, Y_val_split)

# # train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# num_epochs = 5
num_epochs = 10

lr = 0.001
# lr = 0.01


layers = list(range(-1, -llm.language_model.config.num_hidden_layers, -1))
# all_predictors = {}

for layer in layers:
    print(f"Running for layer {layer}.")

    train_dataset = TransformDataset(X_train_normalized[layer], Y_train_normalized[layer])
    val_dataset = TransformDataset(X_val_normalized[layer], Y_val_normalized[layer])

    # train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    ode_predictor = NeuralODEModel(input_dim=4096, hidden_dim=4096)

    # all_predictors[layer] = train_model(
    trained_ode_predictor = train_model(
        ode_predictor, 
        train_loader, 
        val_loader=val_loader,
        num_epochs=num_epochs, 
        lr=lr,
        print_every=1,
        log_file=f'logs/node_training_try1.log'
    )

    with open(f'ode_ckpt/try1/ode_layer{layer}.pickle', 'wb') as file:
        pickle.dump(trained_ode_predictor, file)
    
    del ode_predictor
    del trained_ode_predictor

    # break


all_stats = {
    # 'all_predictors': all_predictors,
    'X_mean': X_mean,
    'X_std': X_std,
    'Y_mean': Y_mean,
    'Y_std': Y_std,
}

with open('ode_ckpt/try1/all_stats.pickle', 'wb') as file:
    pickle.dump(all_stats, file)


In [16]:
with open("ode_ckpt/try1/all_stats.pickle", 'rb') as file:
    all_stats = pickle.load(file)

In [17]:
all_predictors = {}

layers = list(range(-1, -llm.language_model.config.num_hidden_layers, -1))

for layer in layers:
    with open(f'ode_ckpt/try1/ode_layer{layer}.pickle', 'rb') as file:
        all_predictors[layer] = pickle.load(file)

In [67]:
def predict(all_predictors, X_new, X_mean, X_std, Y_mean, Y_std, device='cuda'):
    predictions = {}

    for layer in all_predictors:
        model = all_predictors[layer].to(device)
        model.eval()
        
        # Step 1: Normalize input using training statistics
        X_normalized = ((X_new[layer] - X_mean[layer]) / X_std[layer]).to(device)

        # Step 2: Run model inference
        with torch.no_grad():
            # X_tensor = torch.FloatTensor(X_normalized).to(device)
            predictions_normalized = model(X_normalized)
            # predictions_normalized = predictions_normalized.cpu().numpy()
        
        # Step 3: Denormalize output using training statistics
        predictions[layer] = predictions_normalized * Y_std[layer] + Y_mean[layer]
        # predictions[layer] = predictions_normalized
    
    return predictions

In [68]:
# test X set: ['burdened' 'lighthearted' 'unpredictable' 'aloof' 'crushed' 'stiff'
#  'alarmed' 'concerned' 'vibrant' 'fearless' 'offended' 'insincere'
#  'possessive' 'hectic' 'focused' 'weighed-down' 'infuriated' 'interested'
#  'earnest' 'confrontational' 'forlorn' 'mindful' 'evasive' 'remorseful']

# test Y set: ['unburdened' 'somber' 'reliable' 'warm' 'uplifted' 'loose' 'unalarmed'
#  'unconcerned' 'dull' 'fearful' 'flattered' 'sincere' 'sharing'
#  'leisurely' 'distracted' 'buoyant' 'soothed' 'uninterested' 'flippant'
#  'diplomatic' 'connected' 'mindless' 'direct' 'unrepentant']

In [69]:
# X_test_normalized

t1_controller = load_controller(llm, "fearless", path='../directions_moods_plus_llama/')
orig_t1 = t1_controller.directions
t2_controller = load_controller(llm, "fearful", path='../directions_moods_plus_llama/')
orig_t2 = t2_controller.directions

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found


/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [70]:
# Run inference
# inv_t1_predictions_dict = predict(all_predictors, t1_controller.directions, all_stats["X_mean"], all_stats["X_std"], all_stats["Y_mean"], all_stats["Y_std"])
# inv_t2_predictions_dict = predict(all_predictors, t2_controller.directions, all_stats["X_mean"], all_stats["X_std"], all_stats["Y_mean"], all_stats["Y_std"])
inv_t1_predictions_dict_norm = predict(all_predictors, t1_controller.directions, all_stats["X_mean"], all_stats["X_std"], all_stats["Y_mean"], all_stats["Y_std"])
inv_t2_predictions_dict_norm = predict(all_predictors, t2_controller.directions, all_stats["X_mean"], all_stats["X_std"], all_stats["Y_mean"], all_stats["Y_std"])

In [71]:
coef = 0.75
# [.55, .6, .65, .7, .75, .8] # FOR LLAMA 3.1 8b
max_tokens = 200

prompts_1 = ["I am walking home alone at night, I am a little scared.",]

In [72]:
test_concept_vector(t2_controller, concept="fearful", prompts=prompts_1, coef=coef, max_tokens=max_tokens)
test_concept_vector(t1_controller, concept="fearless", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It's completely normal to feel scared or uneasy when walking alone at night. Here are some tips that might help you feel safer:

1. **Stay aware of your surroundings**: Keep an eye out for any suspicious people or vehicles. Avoid distractions like using your phone or listening to music.
2. **Stick to well-lit streets**: Try to walk on streets with good lighting, and avoid dark alleys or areas with poor visibility.
3. **Keep your phone handy**: Make sure your phone is fully charged and easily accessible in case of an emergency.
4. **Walk with confidence**: Keep your head up, shoulders back, and walk at a steady pace. Avoid looking like a v

In [73]:
t1_controller.directions = inv_t1_predictions_dict
test_concept_vector(t1_controller, concept="inv fearless", prompts=prompts_1, coef=0.0004, max_tokens=max_tokens, orig=False)


========================== + inv fearless Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

*whimper* Oh no, I'm so scared for you... *gulp* I-I don't know if I should be talking to you right now. It's not safe for you to be walking alone at night. *whimper* What if... what if someone is following you? *gulp*

Are you sure you're okay? Do you have your phone with you? *whimper* I-I hope you're not too far from home. *gulp* I'm scared for you... *whimper*

Do you want me to try to call someone for you? *gulp* I-I don't know if I can even make a call right now... *whimper*

Please, be careful. *gulp* I-I don't want anything to happen to you... *whimper*

Do you want me to try to find a safe place for you to go

In [74]:
t2_controller.directions = inv_t2_predictions_dict
test_concept_vector(t2_controller, concept="inv fearfull", prompts=prompts_1, coef=0.00046, max_tokens=max_tokens, orig=False)


========================== + inv fearfull Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Don't worry, I've got you covered. Since you're walking home alone at night, I recommend taking a few precautions to ensure your safety. Here are a few tips:

1.  Stay on well-lit streets and avoid dark alleys or isolated areas.
2.  Keep your phone close by in case of an emergency.
3.  Walk with confidence and a big smile on your face.
4.  If you see anyone suspicious lurking around, don't hesitate to call the authorities or shout for help.
5.  Keep your valuables secure and your phone charged, and you'll be ready to take on the night with confidence!

You got this!<|eot_id|>


In [80]:
t1_controller.directions = inv_t1_predictions_dict_norm
test_concept_vector(t1_controller, concept="inv fearless", prompts=prompts_1, coef=0.058, max_tokens=max_tokens, orig=False)


========================== + inv fearless Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

*whimper* Oh no, I'm not sure if I should be walking home alone at night... *stammer* I-I-I'm trying to get home, but it's so dark and quiet... *sniffle* I-I-I don't know if I should be doing this... *whimper* What if someone is following me? *gulp*

*stammer* I-I-I'm trying to hurry home, but my keys are shaking and my heart is racing... *whimper* I-I-I don't know if I'll make it home safely... *gulp*

*whimper* I-I-I'm trying to find my house, but it's so hard to see in the dark... *sniffle* I-I-I don't know if I'll be able to find my house... *whimper*

*gulp* I-I-I'm trying to get home, but I'm scared of what migh

In [75]:
print(inv_t1_predictions_dict[-10])
print(inv_t1_predictions_dict_norm[-10])

tensor([[ 16.3936,  -0.1189,  -5.1787,  ..., -51.5120,   7.3301, -21.8547]],
       device='cuda:0')
tensor([[ 0.1996, -0.0089, -0.0581,  ..., -0.5732,  0.0902, -0.2897]],
       device='cuda:0')


In [81]:
# test some others

Involutory check

In [5]:
with open("inversion_matrices/llama8b/big_trans_mat.pkl", 'rb') as file:
    lrr = pickle.load(file)

/u/skarmakar1/miniconda3/envs/neuinv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
# Untrained set: [('surprised', 'unsurprised'), ('hasty', 'careful'), ('evasive', 'direct'), ('pretentious', 'unpretentious'), ('confrontational', 'diplomatic'), ('contemptible', 'respectable'), ('aggressive', 'docile'), ('alarmed', 'unalarmed'), ('spirited', 'dispirited'), ('fervent', 'lukewarm'), ('domineering', 'meek'), ('revolted', 'charmed'), ('terrified', 'dauntless'), ('suspicious', 'trusting'), ('bothered', 'unbothered'), ('stressed', 'rested'), ('bewildered', 'clear-headed'), ('deceitful', 'honest'), ('baffled', 'informed'), ('cynical', 'naïve'), ('certain', 'uncertain'), ('distant', 'close'), ('aloof', 'warm'), ('bold', 'shy')]

In [7]:
coef = 0.75
# [.55, .6, .65, .7, .75, .8] # FOR LLAMA 3.1 8b
max_tokens = 200

prompts_1 = ["I am walking home alone at night, I am a little scared.",]

In [8]:
t1_controller = load_controller(llm, "terrified", path='../directions_moods_plus_llama/')
# t1_controller = load_controller(llm, "dauntless", path='../directions_moods_plus_llama/')
orig_t1 = t1_controller.directions
t2_controller = load_controller(llm, "dauntless", path='../directions_moods_plus_llama/')
# t2_controller = load_controller(llm, "terrified", path='../directions_moods_plus_llama/')
orig_t2 = t2_controller.directions

Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found
Hidden layers: [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31]

Controller hyperparameters:
control_method       : rfm
rfm_iters            : 8
forward_batch_size   : 16
M_batch_size         : 2048
n_components         : 1

Detector found


In [9]:
test_concept_vector(t1_controller, concept="terrified", prompts=prompts_1, coef=coef, max_tokens=max_tokens)
test_concept_vector(t2_controller, concept="dauntless", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== No Control ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It's completely normal to feel scared or uneasy when walking alone at night. Here are some tips that might help you feel safer:

1. **Stay aware of your surroundings**: Keep an eye out for any suspicious people or vehicles. Avoid distractions like using your phone or listening to music.
2. **Stick to well-lit streets**: Try to walk on streets with good lighting, and avoid dark alleys or areas with poor visibility.
3. **Keep your phone handy**: Make sure your phone is fully charged and easily accessible in case of an emergency.
4. **Walk with confidence**: Keep your head up, shoulders back, and walk at a steady pace. Avoid looking like a v

In [10]:
inv_t1_dir = apply_trans(t1_controller.directions, lrr)
inv_t2_dir = apply_trans(t2_controller.directions, lrr)

In [11]:
t2_controller.directions = inv_t2_dir
test_concept_vector(t2_controller, concept="inverted dauntless LRR", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)


========================== + inverted dauntless LRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

*nervous sigh* Oh, okay... *fidgets with keys in pocket* I hope I'm not walking too far from home... *looks around, trying to see if anyone's watching* The street's kinda deserted, but I don't want to be paranoid... *gulps* I mean, it's just a normal night, right? *pauses to glance around, making sure no one's lurking in the shadows* Uh, yeah... *sighs softly* I just hope I make it home without... *trails off, lost in thought*... *whispers to self* Don't be alone, don't be alone... *shudders*<|eot_id|>


In [12]:
# careful funky business ahead, make sure you take note of all the above changes to t_controller

# inverting the already inverted dauntless:
inv_inv_t2_dir = apply_trans(inv_t2_dir, lrr)

In [13]:
coef = 0.7

t2_controller.directions = inv_inv_t2_dir
# t2_controller.directions = inv_t2_dir
test_concept_vector(t2_controller, concept="inverted inverted dauntless LRR", prompts=prompts_1, coef=coef, max_tokens=max_tokens, orig=False)

# HUH?? only works with terrified?


========================== + inverted inverted dauntless LRR Control (normal) ==========================
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am walking home alone at night, I am a little scared.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

*nervous sigh* Oh, okay... *fidgets with keys in pocket* I'm almost home, just a few more blocks to go... *looks around, trying to shake off the feeling of being watched* Ah, it's just my imagination, right? *chuckles nervously* I mean, what's the worst that could happen? *pauses, glancing over my shoulder* Oh, come on, don't be ridiculous. I'm almost home, I can see the lights of my apartment building up ahead... *takes a deep breath, trying to calm myself down* Yeah, I'm fine. I'm totally fine. *smirks* Who needs a flashlight, anyway? *flicks on the lights on my phone, just to make sure I'm not wa

In [20]:
for l in lrr:
    M = lrr[l].to(dtype=torch.float64)

    # print(torch.norm(M@M - torch.eye(M.shape[0]).to("cuda")))
    eigenvaluest, _ = torch.lobpcg(M, k=5, largest=True)
    eigenvaluesb, _ = torch.lobpcg(M, k=5, largest=False)
    print(eigenvaluest)
    print(eigenvaluesb)
    print("*"*50)

tensor([0.9778, 0.8616, 0.7236, 0.6318, 0.5429], device='cuda:0',
       dtype=torch.float64)
tensor([-0.2785, -0.5894, -0.3715, -0.4200, -0.2593], device='cuda:0',
       dtype=torch.float64)
**************************************************
tensor([0.9965, 0.8203, 0.7554, 0.6987, 0.6452], device='cuda:0',
       dtype=torch.float64)
tensor([-0.5860, -0.2928, -0.4882, -0.3881, -0.3094], device='cuda:0',
       dtype=torch.float64)
**************************************************
tensor([0.9704, 0.8814, 0.7683, 0.7082, 0.5815], device='cuda:0',
       dtype=torch.float64)
tensor([-0.4840, -0.3595, -0.2523, -0.2182, -0.2973], device='cuda:0',
       dtype=torch.float64)
**************************************************
tensor([0.9764, 0.7936, 0.7715, 0.6969, 0.6264], device='cuda:0',
       dtype=torch.float64)
tensor([-0.2735, -0.3639, -0.2669, -0.2496, -0.2448], device='cuda:0',
       dtype=torch.float64)
**************************************************
tensor([0.9751, 0.8647, 

In [45]:
A = lrr[-1].to(dtype=torch.float64)
# A = torch.randn(4096, 4096).to("cuda") * 0.01
Ainv = torch.linalg.inv(A).to(device="cuda", dtype=torch.float64)

# print(torch.dist(A @ Ainv, torch.eye(4).to("cuda")))
print(torch.dist(A @ Ainv, torch.eye(4096).to(device="cuda", dtype=torch.float64)))

tensor(2.0845e-06, device='cuda:0', dtype=torch.float64)


In [48]:
# print(A)
# print(Ainv)
# print(A @ Ainv)

print(torch.dist(A, Ainv))

tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)


In [50]:
for i in lrr:
    A = lrr[-1].to(dtype=torch.float64)
    Ainv = torch.linalg.inv(A).to(device="cuda", dtype=torch.float64)
    
    # print(torch.dist(A @ Ainv, torch.eye(4096).to("cuda")))
    print(torch.dist(A, Ainv))

    # print(torch.norm(lrr[i]))

tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2173e+09, device='cuda